In [1]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.4/491.4 kB 28.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 16.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 12.8 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2025.3.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which 

In [2]:
from datasets import load_dataset
raw_dataset = load_dataset('kde4',lang1='en',lang2='fr')

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/5.10k [00:00<?, ?B/s]

kde4.py:   0%|          | 0.00/4.25k [00:00<?, ?B/s]

The repository for kde4 contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/kde4.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


Generating train split:   0%|          | 0/210173 [00:00<?, ? examples/s]

In [3]:
raw_dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'translation'],
        num_rows: 210173
    })
})

In [4]:
### let's split
split_dataset=raw_dataset['train'].train_test_split(train_size=.9,seed=20)
split_dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'translation'],
        num_rows: 189155
    })
    test: Dataset({
        features: ['id', 'translation'],
        num_rows: 21018
    })
})

In [5]:
split_dataset['validation']=split_dataset.pop('test')

In [6]:
split_dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'translation'],
        num_rows: 189155
    })
    validation: Dataset({
        features: ['id', 'translation'],
        num_rows: 21018
    })
})

In [7]:
split_dataset['train']['translation'][1]

{'en': 'Default to expanded threads',
 'fr': 'Par défaut, développer les fils de discussion'}

In [8]:
dataset = split_dataset['train']

for example in dataset:
    en_text = example['translation']['en']
    fr_text = example['translation']['fr']


    if 'email' in en_text.lower():
        print("English:", en_text)
        print("French:", fr_text)
        break


English: Sends the chart as an email attachment.
French: Envoie le diagramme comme pièce jointe.


In [9]:
from transformers import AutoTokenizer
model_id="Helsinki-NLP/opus-mt-en-fr"
tokenizer=AutoTokenizer.from_pretrained(model_id,return_tensors='pt')

tokenizer_config.json:   0%|          | 0.00/42.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.42k [00:00<?, ?B/s]

source.spm:   0%|          | 0.00/778k [00:00<?, ?B/s]

target.spm:   0%|          | 0.00/802k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.34M [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


In [10]:
en_sentence = split_dataset["train"][1]["translation"]["en"]
fr_sentence = split_dataset["train"][1]["translation"]["fr"]

inputs = tokenizer(en_sentence, text_target=fr_sentence)
inputs

{'input_ids': [47591, 12, 9842, 19634, 9, 0], 'attention_mask': [1, 1, 1, 1, 1, 1], 'labels': [577, 5891, 2, 3184, 16, 2542, 5, 1710, 0]}

In [11]:
max_length=128
def preprocess_function(examples):
  inputs=[ex['en'] for ex in examples['translation']]
  targets=[ex['fr'] for ex in examples['translation']]
  model_inputs=tokenizer(
      inputs,
      text_target=targets,
      max_length=max_length,
      truncation=True
  )
  return model_inputs

In [12]:
tokenized_datasets = split_dataset.map(
    preprocess_function,
    batched=True,
    remove_columns=split_dataset["train"].column_names,
)

Map:   0%|          | 0/189155 [00:00<?, ? examples/s]

Map:   0%|          | 0/21018 [00:00<?, ? examples/s]

In [13]:
from transformers import AutoModelForSeq2SeqLM

model = AutoModelForSeq2SeqLM.from_pretrained(model_id)

pytorch_model.bin:   0%|          | 0.00/301M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

In [14]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [15]:
!pip install sacrebleu

model.safetensors:   0%|          | 0.00/301M [00:00<?, ?B/s]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.1/104.1 kB 9.2 MB/s eta 0:00:00


In [16]:
!pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 7.3 MB/s eta 0:00:00


In [17]:
import evaluate

metric = evaluate.load("sacrebleu")

In [18]:
import numpy as np
def compute_metrics(eval_preds):
  preds , labels = eval_preds
  if isinstance(preds,tuple):
    preds=preds[0]
  decoded_preds=tokenizer.batch_decode(preds,skip_special_tokens=True)
  labels = np.where(labels!=-100,labels,tokenizer.pad_token_id)
  decoded_labels=tokenizer.batch_decode(labels,skip_special_tokens=True)
  decoded_preds = [pred.strip() for pred in decoded_preds]
  decoded_labels = [[label.strip()] for label in decoded_labels]

  result = metric.compute(predictions=decoded_preds, references=decoded_labels)
  return {"bleu": result["score"]}


In [19]:
from huggingface_hub import notebook_login

notebook_login()

In [21]:
from transformers import Seq2SeqTrainingArguments

args = Seq2SeqTrainingArguments(
    f"marian-finetuned-kde4-en-to-fr",

    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=64,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=3,
    predict_with_generate=True,
    fp16=True,
    push_to_hub=True,
)

In [22]:
from transformers import Seq2SeqTrainer

trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

<ipython-input-22-1468824c6e77>:3: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


In [23]:
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: alireza2002hey (alireza2002hey-shiraz-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Step,Training Loss
500,1.386400
1000,1.230700
1500,1.195500
2000,1.137600
2500,1.111000
3000,1.079100
3500,1.044200
4000,1.032000
4500,1.039100
5000,1.010000


/usr/local/lib/python3.11/dist-packages/transformers/modeling_utils.py:3339: UserWarning: Moving the following attributes in the config to the generation config: {'max_length': 512, 'num_beams': 4, 'bad_words_ids': [[59513]]}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(


TrainOutput(global_step=17736, training_loss=0.9368094494604129, metrics={'train_runtime': 3310.6134, 'train_samples_per_second': 171.408, 'train_steps_per_second': 5.357, 'total_flos': 1.1353213104095232e+16, 'train_loss': 0.9368094494604129, 'epoch': 3.0})

In [24]:
trainer.evaluate(max_length=max_length)

{'eval_loss': 0.8553687334060669,
 'eval_bleu': 32.66555156176086,
 'eval_runtime': 1417.8321,
 'eval_samples_per_second': 14.824,
 'eval_steps_per_second': 0.232,
 'epoch': 3.0}

In [25]:
trainer.push_to_hub(tags="translation", commit_message="Training complete")

events.out.tfevents.1746600647.82ba81c99904.2181.1:   0%|          | 0.00/413 [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/Alireza0017/marian-finetuned-kde4-en-to-fr/commit/7a862d6efbade80188085a65768a9cc12f1926e2', commit_message='Training complete', commit_description='', oid='7a862d6efbade80188085a65768a9cc12f1926e2', pr_url=None, repo_url=RepoUrl('https://huggingface.co/Alireza0017/marian-finetuned-kde4-en-to-fr', endpoint='https://huggingface.co', repo_type='model', repo_id='Alireza0017/marian-finetuned-kde4-en-to-fr'), pr_revision=None, pr_num=None)